# Step 1) Data Preparation

In [ ]:
algo_name='algo_sma'

In [ ]:
s3bucket=!(aws cloudformation list-exports --query "Exports[?Name=='algotrading-s3bucket'].Value" --output text)
s3bucket=s3bucket[0]
s3bucket

In [ ]:
import pandas as pd
from pyathena import connect
conn = connect(s3_staging_dir='s3://'+s3bucket+'/results/')

df = pd.read_sql("SELECT dt,open,high,low,close,vol FROM algo_data.hist_data_daily;", conn)
df.set_index(pd.DatetimeIndex(df['dt']),inplace=True)
del df['dt']
df.head()

In [ ]:
from pathlib import Path
trainCount=int(len(df)*0.4)
dfTrain = df.iloc[:trainCount]
dfTest = df.iloc[trainCount:]

dfTest.to_csv('/opt/ml/input/data/training/data.csv')
dfTest.head()

In [ ]:
%matplotlib inline
dfTest["close"].plot()

# Step 2) Modify Strategy Configuration 

In the following cell, you can adjust the parameters for the strategy.

* `fast_period` = Fast Period for Moving Average Indicator in min (e.g. 8)
* `slow_period` = Slow Period for Moving Average Indicator in min (e.g. 21)
* `size` = The number of shares for a transaction (e.g. 100)

In [ ]:
%%writefile /opt/ml/input/config/hyperparameters.json
{ "fast_period" : "8",
  "slow_period" : "21",
  "size" : "100"
}

In [ ]:
%run /opt/program/update_config.py $algo_name $s3bucket

# Step 3) Modify Strategy Code

Here are some helpful links:
* Backtrader Documentation: https://www.backtrader.com/docu/strategy/
* TA-Lib Indicator Reference: https://www.backtrader.com/docu/talibindautoref/
* Backtrader Indicator Reference: https://www.backtrader.com/docu/indautoref/

In [ ]:
%%writefile /opt/program/{algo_name}.py
import backtrader as bt
from algo_base import *
import pytz
from pytz import timezone

class MyStrategy(StrategyTemplate):

    def __init__(self):  # Initiation
        super(MyStrategy, self).__init__()
        self.config["fast_period"]=int(self.config["fast_period"])
        self.config["slow_period"]=int(self.config["slow_period"])
        self.config["size"]=int(self.config["size"])
        print(self.config)
        self.emaFast = bt.ind.ExponentialMovingAverage(period=self.config["fast_period"])
        self.emaSlow = bt.ind.ExponentialMovingAverage(period=self.config["slow_period"])
        self.size = self.config["size"]

    def init_broker(broker):
        broker.setcash(100000.0)
        broker.setcommission(commission=0.0) 
        
    def add_data(cerebro):
        data = btfeeds.GenericCSVData(
            dataname=MyStrategy.TRAIN_FILE,
            dtformat=('%Y-%m-%d'),
            timeframe=bt.TimeFrame.Days,
            datetime=0,
            time=-1,
            high=2,
            low=3,
            open=1,
            close=4,
            volume=5,
            openinterest=-1
        )
        cerebro.adddata(data)

    def next(self):  # Processing
        super(MyStrategy, self).next()
        dt=self.datas[0].datetime.datetime(0)
        if not self.position:
            if self.emaFast[0] > self.emaSlow[0]:
                self.buy(size=self.size) # Go long
            else:
                self.sell(size=self.size) # Go short
        elif self.position.size>0 and self.emaFast[0] < self.emaSlow[0]:
            self.sell(size=2*self.size) # Go short
        elif self.position.size<0 and self.emaFast[0] > self.emaSlow[0]:          
            self.buy(size=2*self.size) # Go long

# Step 4) Backtest Locally

In [ ]:
%run /opt/program/train

# Step 5) Backtest Remotely with SageMaker

In [ ]:
!aws s3 cp "/opt/program/" "s3://{s3bucket}/{algo_name}/" --recursive --exclude "*" --include "{algo_name}*.*"

In [ ]:
conf_file='/opt/ml/input/config/hyperparameters.json'
with open(conf_file, 'r') as f:
    config = json.load(f)
config['s3']=s3_bucket    
config['chart']='true'
print(config)

In [ ]:
#Run Remote Backtest via SageMaker
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator 

role = get_execution_role()
sess = sage.Session()

WORK_DIRECTORY = '/opt/ml/input/data/training'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix='data')
print(data_location)

prefix=algo_name
job_name=prefix.replace('_','-')

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/algotrading:1.0'

algo = sage.estimator.Estimator(
    image_uri=image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
    base_job_name=job_name,
    hyperparameters=config,
    metric_definitions=[
        {
            "Name": "algo:pnl",
            "Regex": "Total PnL:(.*?)]"
        },
        {
            "Name": "algo:sharpe_ratio",
            "Regex": "Sharpe Ratio:(.*?),"
        }
    ])

In [ ]:
algo.fit(data_location)

In [ ]:
#Get Algo Metrics
from sagemaker.analytics import TrainingJobAnalytics

latest_job_name = algo.latest_training_job.job_name
metrics_dataframe = TrainingJobAnalytics(training_job_name=latest_job_name).dataframe()
metrics_dataframe

In [ ]:
#Get Algo Chart from S3
model_name=algo.model_data.replace('s3://'+sess.default_bucket()+'/','')
import boto3
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sess.default_bucket())
my_bucket.download_file(model_name,'model.tar.gz')
!tar -xzf model.tar.gz
!rm model.tar.gz
from IPython.display import Image
Image(filename='chart.png') 

# Step 6) Run Hyperparameter Optimization with SageMaker

In [ ]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

hyperparameter_ranges = {
    "fast_period": IntegerParameter(5, 10),
    "slow_period": IntegerParameter(21, 31)
}
objective_metric_name= "algo:pnl"
tuner = HyperparameterTuner(algo,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=6,
    max_parallel_jobs=3,
    metric_definitions=[
        {
            "Name": "algo:pnl",
            "Regex": "Total PnL:(.*?)]"
        }
    ]
   )

In [ ]:
tuner.fit(data_location)

In [ ]:
best_params=boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['BestTrainingJob']['TunedHyperParameters']
best_params

In [ ]:
from sagemaker.analytics import TrainingJobAnalytics
bestjob=tuner.best_training_job()
metrics_dataframe = TrainingJobAnalytics(training_job_name=bestjob).dataframe()
metrics_dataframe

# Step 7) Backtest Locally with Optimal Parameters

In [ ]:
# Use optimal hyperparameter and test data
conf_file='/opt/ml/input/config/hyperparameters.json'
with open(conf_file, 'r') as f:
    config = json.load(f)
config['fast_period']=best_params['fast_period']
config['slow_period']=best_params['slow_period']
config['chart']='false'
print(config)

In [ ]:
%run /opt/program/train

### Congratulations! You've completed this strategy.